In [ ]:
# Installation des dépendances
!pip install cudf-cu11 cuml-cu11 dask-cudf-cu11
!pip install requests

import cudf
import cuml
from cuml.linear_model import LogisticRegression
from cuml.ensemble import RandomForestClassifier
from cuml.metrics import accuracy_score, recall_score, precision_score, roc_auc_score
import requests
import pandas as pd
import time

# Téléchargement des données
!wget https://votre-url/creditcard.csv -O creditcard.csv

# Fonction d'entraînement RAPIDS
def train_rapids():
    # Chargement des données
    df = cudf.read_csv('creditcard.csv')
    
    # Préparation
    features = [col for col in df.columns if col not in ("Class",)]
    X = df[features]
    y = df["Class"]
    
    # Split
    X_train, X_test, y_train, y_test = cuml.model_selection.train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    
    results = {}
    
    # LogisticRegression
    start_time = time.time()
    lr = LogisticRegression()
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_test)
    y_proba = lr.predict_proba(X_test)[:, 1]
    train_time = time.time() - start_time
    
    results["RAPIDS-GPU-LogisticRegression"] = {
        "auc": float(roc_auc_score(y_test, y_proba)),
        "accuracy": float(accuracy_score(y_test, y_pred)),
        "recall": float(recall_score(y_test, y_pred)),
        "precision": float(precision_score(y_test, y_pred)),
        "training_time": train_time
    }
    
    return results

# Entraînement et envoi des résultats
results = train_rapids()

# Sauvegarde locale
pd.DataFrame([results]).to_csv('resultats_auc_rapids.csv', index=False)

# Envoi au service de comparaison
response = requests.post(
    'http://votre-service-compare/upload-results',
    json=results
)
print("Résultats envoyés:", response.status_code)